In [40]:
from algorithms.persistance import load_from_json

import os
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

In [41]:
def convert_to_df(path: str):
    results = load_from_json(path)
    config_list = []
    performance_list = []
    for result in results:
        config_list.append(result['config'])
        performance_list.append(result['performance'])
    config = pd.DataFrame(config_list)
    performance = pd.DataFrame(performance_list)
    result_list = pd.concat([performance, config], axis=1)
    return result_list

# show exerpt of json file as df

In [93]:
de = 'lstm'
path = f'../persistent_data/{de}.json'
result_list = convert_to_df(path)
result_list

,true_positives,false_positives,true_negatives,false_negatives,alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,recall,precision_with_cfa,precision_with_syscalls,scenario,ngram,batch_size,embedding_size,return_value,thread_change_flag,time_delta,detection_time
0,173,1,1229731,194416,172,174,0.988506,1,0,0.988506,0.994220,0.994220,CVE-2017-7529/,2,1024,4,False,False,False,14.714413
1,0,0,1226104,194589,0,174,0.000000,0,0,0.000000,0.000000,0.000000,CVE-2017-7529/,6,1024,4,False,False,False,15.412062
2,0,14,2836396,285051,0,100,0.000000,14,0,0.000000,0.000000,0.000000,CVE-2014-0160/,2,1024,4,False,False,False,33.687482
3,1,0,2833010,285050,1,100,0.010000,0,0,0.010000,1.000000,1.000000,CVE-2014-0160/,6,1024,4,False,False,False,34.964247
4,23,17,2668699,2095470,18,98,0.183673,15,2,0.183673,0.514286,0.514286,Bruteforce_CWE-307/,6,1024,4,False,False,False,41.704447
5,18,50,2672034,2095475,14,98,0.142857,45,4,0.142857,0.222222,0.218750,Bruteforce_CWE-307/,2,1024,4,False,False,False,51.022163
6,0,5,3398412,1474597,0,155,0.000000,5,0,0.000000,0.000000,0.000000,CVE-2012-2122/,2,1024,4,False,False,False,53.987786
7,10,17,3393820,1474587,10,155,0.064516,16,1,0.064516,0.370370,0.370370,CVE-2012-2122/,6,1024,4,False,False,False,54.603340
8,346,1,1222475,194243,172,174,0.988506,1,0,0.988506,0.994220,0.994220,CVE-2017-7529/,10,1024,4,False,False,False,12.700604
9,20355,64,2665284,2075138,92,98,0.938776,60,4,0.938776,0.589744,0.589744,Bruteforce_CWE-307/,10,1024,4,False,False,False,52.002842


# connect to db and create table

In [95]:
conn = sql.connect('evaluation.db')
try:
    result_list.to_sql(de, conn)
except ValueError:
    print('Table already exists')

Table already exists


# show all different configs DR > 0.5

In [112]:
conn = sql.connect('evaluation.db')
result = pd.read_sql('SELECT \
                             ngram, \
                             embedding_size as embed, \
                             return_value as rv, \
                             thread_change_flag as tcf, \
                             time_delta as td, \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             ROUND(AVG(detection_rate),2) as DR, \
                             COUNT(*) \
                     FROM lstm \
                     GROUP BY ngram, \
                              embedding_size, \
                              rv, \
                              tcf, \
                              td \
                     HAVING ngram > 8 \
                     ORDER BY  \
                              DR desc, \
                              fp_sum desc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,ngram,embed,rv,tcf,td,fp_norm,fp_ex,fp_sum,DR,COUNT(*)
0,10,8,0,1,1,19.25,0.50,158,0.73,8
1,10,4,1,1,1,18.00,1.13,153,0.71,8
2,10,4,0,1,0,4.88,0.25,41,0.71,8
3,10,4,1,0,0,13.75,0.63,115,0.70,8
4,10,4,1,1,0,12.00,0.63,101,0.70,8
5,10,4,0,0,0,14.88,0.75,125,0.67,8
6,10,4,0,1,1,13.50,0.50,112,0.63,8
7,10,6,1,1,1,9.50,0.00,76,0.63,8
8,10,4,0,0,1,9.13,0.25,75,0.63,8
9,10,8,0,1,0,19.25,0.88,161,0.62,8


# comparison with and without time_delta 

In [87]:
conn = sql.connect('evaluation.db')
result = pd.read_sql('SELECT scenario, time_delta as td,  \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             AVG(detection_rate) as DR \
                             COUNT(*) \
                      FROM lstm \
                      GROUP BY scenario, td \
                      ORDER BY scenario, DR desc, fp_sum asc', conn)
result

,scenario,td,fp_norm,fp_ex,fp_sum,DR
0,Bruteforce_CWE-307/,1,27.53,1.75,1054,0.324546
1,Bruteforce_CWE-307/,0,20.39,1.39,893,0.251867
2,CVE-2012-2122/,1,9.75,0.44,367,0.039606
3,CVE-2012-2122/,0,11.80,0.49,504,0.018725
4,CVE-2014-0160/,1,6.31,0.17,233,0.011111
5,CVE-2014-0160/,0,4.59,0.12,193,0.005854
6,CVE-2017-7529/,1,1.50,0.00,54,0.906290
7,CVE-2017-7529/,0,0.83,0.00,34,0.868517
8,CVE-2018-3760/,1,9.72,0.42,365,1.000000
9,CVE-2018-3760/,0,9.41,0.63,412,1.000000


# comparison with and without return_value

In [91]:
conn = sql.connect('evaluation.db')
result = pd.read_sql('SELECT scenario, return_value as rv,  \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             AVG(detection_rate) as DR \
                      FROM lstm \
                      GROUP BY scenario, rv \
                      ORDER BY scenario, DR desc, fp_sum asc', conn)
result

,scenario,rv,fp_norm,fp_ex,fp_sum,DR
0,Bruteforce_CWE-307/,1,24.13,1.44,997,0.311094
1,Bruteforce_CWE-307/,0,23.32,1.68,950,0.259936
2,CVE-2012-2122/,1,13.24,0.58,525,0.039728
3,CVE-2012-2122/,0,8.51,0.36,346,0.017535
4,CVE-2014-0160/,1,7.34,0.24,288,0.011842
5,CVE-2014-0160/,0,3.49,0.05,138,0.004872
6,CVE-2017-7529/,1,0.92,0.00,35,0.910617
7,CVE-2017-7529/,0,1.36,0.00,53,0.862364
8,CVE-2018-3760/,1,9.18,0.26,359,1.000000
9,CVE-2018-3760/,0,9.92,0.79,418,1.000000


# comparison with and without thread_change_flag

In [85]:
conn = sql.connect('evaluation.db')
result = pd.read_sql('SELECT scenario, thread_change_flag as tcf,  \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             AVG(detection_rate) as DR \
                      FROM lstm \
                      GROUP BY scenario, tcf \
                      ORDER BY scenario, DR desc, fp_sum asc', conn)
result

,scenario,tcf,fp_norm,fp_ex,fp_sum,DR
0,Bruteforce_CWE-307/,1,27.86,1.81,1068,0.402494
1,Bruteforce_CWE-307/,0,20.10,1.34,879,0.183425
2,CVE-2012-2122/,1,4.83,0.08,177,0.042652
3,CVE-2012-2122/,0,16.12,0.80,694,0.016050
4,CVE-2014-0160/,1,4.50,0.17,168,0.008333
5,CVE-2014-0160/,0,6.17,0.12,258,0.008293
6,CVE-2017-7529/,1,1.50,0.00,54,0.933589
7,CVE-2017-7529/,0,0.83,0.00,34,0.844547
8,CVE-2018-3760/,1,7.78,0.44,296,1.000000
9,CVE-2018-3760/,0,11.12,0.61,481,1.000000


# comparison of all extra params for each scenario

In [102]:
conn = sql.connect('evaluation.db')
result = pd.read_sql('SELECT scenario, time_delta as td, thread_change_flag as tcf, return_value as rv,  \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             AVG(detection_rate) as DR, \
                      FROM lstm \
                      GROUP BY scenario, td, tcf, rv \
                      ORDER BY scenario, DR desc, fp_sum asc', conn)
result

,scenario,td,tcf,rv,fp_norm,fp_ex,fp_sum,DR,COUNT(*)
0,Bruteforce_CWE-307/,1,1,1,33.11,2.00,316,0.501134,9
1,Bruteforce_CWE-307/,0,1,1,30.00,2.00,288,0.386621,9
2,Bruteforce_CWE-307/,0,1,0,19.44,1.33,187,0.365079,9
3,Bruteforce_CWE-307/,1,1,0,28.89,1.89,277,0.357143,9
4,Bruteforce_CWE-307/,1,0,1,23.33,0.89,218,0.278912,9
5,Bruteforce_CWE-307/,0,0,0,20.73,1.36,243,0.175325,11
6,Bruteforce_CWE-307/,1,0,0,24.78,2.22,243,0.160998,9
7,Bruteforce_CWE-307/,0,0,1,13.58,1.00,175,0.136054,12
8,CVE-2012-2122/,1,1,1,5.67,0.11,52,0.121864,9
9,CVE-2012-2122/,0,1,0,6.78,0.00,61,0.031541,9


In [105]:
result = pd.read_sql('SELECT \
                             scenario, \
                             ngram, \
                             embedding_size as embed, \
                             ROUND(AVG(consecutive_false_positives_normal),2) as fp_norm, \
                             ROUND(AVG(consecutive_false_positives_exploits),2) as fp_ex, \
                             SUM(consecutive_false_positives_normal + consecutive_false_positives_exploits) as fp_sum, \
                             ROUND(AVG(detection_rate),2) as DR, \
                             COUNT(*) \
                     FROM lstm \
                     GROUP BY scenario, \
                              ngram, \
                              embedding_size \
                     ORDER BY scenario, \
                              DR desc, \
                              fp_sum asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,scenario,ngram,embed,fp_norm,fp_ex,fp_sum,DR,COUNT(*)
0,Bruteforce_CWE-307/,10,4,31.25,1.88,265,0.59,8
1,Bruteforce_CWE-307/,10,8,25.14,1.86,189,0.45,7
2,Bruteforce_CWE-307/,2,6,44.50,2.63,377,0.37,8
3,Bruteforce_CWE-307/,2,4,48.38,2.25,405,0.36,8
4,Bruteforce_CWE-307/,2,8,30.13,1.38,252,0.31,8
5,Bruteforce_CWE-307/,6,8,9.38,1.13,84,0.22,8
6,Bruteforce_CWE-307/,6,6,11.50,1.25,102,0.20,8
7,Bruteforce_CWE-307/,6,4,14.25,2.00,130,0.18,8
8,Bruteforce_CWE-307/,10,6,11.25,0.75,96,0.10,8
9,Bruteforce_CWE-307/,10,10,7.00,0.50,15,0.06,2


In [79]:
result = pd.read_sql('SELECT * \
                     FROM lstm', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,index,true_positives,false_positives,true_negatives,false_negatives,alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,...,precision_with_cfa,precision_with_syscalls,scenario,ngram,batch_size,embedding_size,return_value,thread_change_flag,time_delta,detection_time
0,0,173,1,1229731,194416,172,174,0.988506,1,0,...,0.994220,0.994220,CVE-2017-7529/,2,1024,4,0,0,0,14.714413
1,1,0,0,1226104,194589,0,174,0.000000,0,0,...,0.000000,0.000000,CVE-2017-7529/,6,1024,4,0,0,0,15.412062
2,2,0,14,2836396,285051,0,100,0.000000,14,0,...,0.000000,0.000000,CVE-2014-0160/,2,1024,4,0,0,0,33.687482
3,3,1,0,2833010,285050,1,100,0.010000,0,0,...,1.000000,1.000000,CVE-2014-0160/,6,1024,4,0,0,0,34.964247
4,4,23,17,2668699,2095470,18,98,0.183673,15,2,...,0.514286,0.514286,Bruteforce_CWE-307/,6,1024,4,0,0,0,41.704447
5,5,18,50,2672034,2095475,14,98,0.142857,45,4,...,0.222222,0.218750,Bruteforce_CWE-307/,2,1024,4,0,0,0,51.022163
6,6,0,5,3398412,1474597,0,155,0.000000,5,0,...,0.000000,0.000000,CVE-2012-2122/,2,1024,4,0,0,0,53.987786
7,7,10,17,3393820,1474587,10,155,0.064516,16,1,...,0.370370,0.370370,CVE-2012-2122/,6,1024,4,0,0,0,54.603340
8,8,346,1,1222475,194243,172,174,0.988506,1,0,...,0.994220,0.994220,CVE-2017-7529/,10,1024,4,0,0,0,12.700604
9,9,20355,64,2665284,2075138,92,98,0.938776,60,4,...,0.589744,0.589744,Bruteforce_CWE-307/,10,1024,4,0,0,0,52.002842
